In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import keras
from keras.layers import Conv2D, Dense, LeakyReLU, Dropout, Input
from keras.layers import Reshape, Conv2DTranspose, Flatten
from keras.layers import BatchNormalization
from keras.models import Model
from keras import optimizers
import warnings
import os
import pandas as pd
from keras.preprocessing import image
from keras.callbacks import ModelCheckpoint, LearningRateScheduler

In [2]:
x_train=[]
x_label=[]
for i in range(6):
    if(i==0):
        continue
    filename=os.listdir("./zkx/simulate/"+str(i)+"/")
    for j in filename:
        df =  pd.read_csv("./zkx/simulate/"+str(i)+"/"+j,header=None)
        txt = df[0].tolist()
        while(len(txt)<400):
            txt.append(0)
        x_label.append(i-1)
        x_train.append(txt)

In [3]:
print(len(x_train))
print(len(x_label))

250
250


In [4]:
y_train=[]
y_label=[]
for i in range(6):
    if(i==0):
        continue
    filename=os.listdir("./zkx/sound/"+str(i)+"/")
    for j in filename:
        df =  pd.read_csv("./zkx/sound/"+str(i)+"/"+j,header=None)
        txt = df[0].tolist()
        while(len(txt)<400):
            txt.append(0)
        y_label.append(i-1)
        y_train.append(txt)

In [5]:
print(len(y_train))
print(len(y_label))

64
64


In [6]:
x_label=keras.utils.to_categorical(x_label)
y_label=keras.utils.to_categorical(y_label)
y_train=np.array(y_train)
y_label=np.array(y_label)
x_train=np.array(x_train)
x_label=np.array(x_label)
x_train=np.expand_dims(x_train,axis=2)
y_train=np.expand_dims(y_train,axis=2)
print(x_train.shape)
print(x_label.shape)
print(y_train.shape)
print(y_label.shape)
x_train=x_train.reshape(250,1,400)
y_train=y_train.reshape(64,1,400)
print(x_train.shape)
print(y_train.shape)

(250, 400, 1)
(250, 5)
(64, 400, 1)
(64, 5)
(250, 1, 400)
(64, 1, 400)


In [7]:
inputB = Input(shape=(1,400))
x=Dense(200,activation='relu')(inputB)
x=keras.layers.MultiHeadAttention(num_heads=3, key_dim=3,value_dim=None)(x,x)
x=Flatten()(x)
x=Dense(128)(x)
x=LeakyReLU()(x)
x=Dropout(0.5)(x)
x=Dense(64)(x)
x=LeakyReLU()(x)
x=Dropout(0.5)(x)
x=Dense(32)(x)
x=LeakyReLU()(x)
x=Dropout(0.5)(x)
x=Dense(5,activation='softmax')(x)
model=Model(inputB,x)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1, 400)]     0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 1, 200)       80200       input_1[0][0]                    
__________________________________________________________________________________________________
multi_head_attention (MultiHead (None, 1, 200)       7427        dense[0][0]                      
                                                                 dense[0][0]                      
__________________________________________________________________________________________________
flatten (Flatten)               (None, 200)          0           multi_head_attention[0][0]   

In [10]:
filepath = "./zkx/best_weights.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=0, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
history = model.fit(x_train,x_label,epochs = 10, shuffle=True,verbose = 1,validation_data=(y_train,y_label),callbacks = callbacks_list)
model.save("./zkx/final.h5")

Epoch 1/10
8/8 [==============================] - 0s 15ms/step - loss: 2.9571 - acc: 0.8880 - val_loss: 1.9224 - val_acc: 0.1875
Epoch 2/10
8/8 [==============================] - 0s 11ms/step - loss: 1.4047 - acc: 0.8880 - val_loss: 1.9472 - val_acc: 0.1875
Epoch 3/10
8/8 [==============================] - 0s 9ms/step - loss: 2.8799 - acc: 0.8720 - val_loss: 1.9689 - val_acc: 0.1875
Epoch 4/10
8/8 [==============================] - 0s 10ms/step - loss: 0.9674 - acc: 0.9560 - val_loss: 1.9620 - val_acc: 0.1875
Epoch 5/10
8/8 [==============================] - 0s 9ms/step - loss: 1.6977 - acc: 0.9120 - val_loss: 1.9036 - val_acc: 0.1875
Epoch 6/10
8/8 [==============================] - 0s 10ms/step - loss: 2.0579 - acc: 0.8960 - val_loss: 1.9629 - val_acc: 0.1875
Epoch 7/10
8/8 [==============================] - 0s 9ms/step - loss: 2.2083 - acc: 0.9000 - val_loss: 1.9823 - val_acc: 0.1875
Epoch 8/10
8/8 [==============================] - 0s 10ms/step - loss: 1.2576 - acc: 0.9280 - val_lo